In [1]:
from tool import *
import plotly.graph_objects as go

# 업체별 선적 실적

# 모니터 2023년 선적 현황

In [65]:
with open('D:/Data/ODM_SR.db', 'rb') as f:
    sr = pickle.load(f)
with open('D:/Data/Shipment Plan_Adjusted_new.DB', 'rb') as f:
    spdb = pickle.load(f)
with open('D:/Data/contact_window.db', 'rb') as f:
    cont = pickle.load(f)
with open('D:/Data/sitemap.db', 'rb') as f:
    site = pickle.load(f)
with open('D:/Data/contact_window.db', 'rb') as f:
    cont = pickle.load(f)

In [66]:
sr = sr.reset_index()
this_week = get_weekname(datetime.datetime.today())
# sr = sr[sr['From Site'].isin(['OS_TCL_CN_P', 'OS_TPV_CN_P', 'OS_AUO_CN_P'])] # 업체 필터링
sr['Week Name'] = sr['Ship Date'].map(get_weekname)
sr[['Year', 'Month']] = sr['Week Name'].apply(lambda x:f'{datetime.date.fromisoformat(x[:10]).isocalendar().year}-{get_month_from_date(datetime.date.fromisoformat(x[:10])):02d}').str.split('-', expand=True)
# sr = sr[sr['Year'] == '2023'] # 2023년 실적만 필터링
sr['Quarter'] = sr['Month'].apply(lambda x:pd.to_datetime(x, format='%m')).dt.to_period('Q').astype(str).apply(lambda x:x[-2:])
sr = sr[sr['Week Name'] <= get_weekname_from(this_week, -1)]
sr['To Site'] = sr['To Site'].replace(ship_to_map)
sr = pd.merge(sr, site[['Region', 'Subsidiary', 'To Site', 'Country']], how='left')

# IT ODM 모델명에서 시리즈명 추출
c1 = sr['From Site'].str.contains('AUO|TPV|TCL') # 모니터 ODM 일 경우
c2 = sr['Mapping Model.Suffix'].str.match('\d{2}[T|Z|U|A][G|D|N|W|C]?\d{2}[A-Z]{1,2}') # PC/태블릿 모델일 경우
c3 = sr['Mapping Model.Suffix'].str.contains('CL600') # 페가트론 CL600 모델일 경우

sr.loc[c1, 'Series'] = sr[c1]['Mapping Model.Suffix'].apply(lambda x:x[:2]) # 모니터 모델의 시리즈명(inch) 추출
sr.loc[c2, 'Series'] = sr[c2]['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).str.replace('D|G|N|W|C', '').apply(lambda x:x[:6]) # PC모델의 대표모델명 추출
sr.loc[c3, 'Series'] = 'CL600'

sr = sr[['Year', 'Month', 'Quarter', 'From Site', 'Division', 'Region', 'Country', 'Subsidiary', 'To Site', 'Series', 'Mapping Model.Suffix', 'Week Name', 'Ship']]
sr = sr.rename(columns={'Ship':'SP'})

C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_1012\3561800162.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
  sr.loc[c2, 'Series'] = sr[c2]['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).str.replace('D|G|N|W|C', '').apply(lambda x:x[:6]) # PC모델의 대표모델명 추출


In [67]:
spdb.groupby(['Planweek', 'Category', 'Created_at']).sum('SP').reset_index().iloc[-16:, ] # 선택할 SP의 Version 목록을 출력

,Planweek,Category,Created_at,SP
86,2023-12-11(W50),Latest26w,202312141259,983191.0
87,2023-12-18(W51),Final26w,202312200913,1048440.0
88,2023-12-18(W51),Final52w,202312191411,1351564.0
89,2023-12-18(W51),Latest26w,202312180928,1035657.0
90,2023-12-18(W51),Latest26w,202312181509,1049245.0
91,2023-12-18(W51),Latest26w,202312190851,1050118.0
92,2023-12-18(W51),Latest26w,202312211119,1047360.0
93,2023-12-25(W52),Final26w,202312271206,1094155.0
94,2023-12-25(W52),Final52w,202312261416,1380956.0
95,2023-12-25(W52),Latest26w,202312251016,1094842.0


In [68]:
i = 99 # 선택할 SP version 설정

In [69]:
sp = spdb.merge(spdb.groupby(['Planweek', 'Category', 'Created_at']).sum('SP').reset_index().iloc[[i], :3])
sp = pd.merge(sp, site[['To Site', 'Subsidiary', 'Country']], how='left')
sp

,Planweek,Created_at,Division,From Site,Region,To Site,Mapping Model.Suffix,Rep PMS,Category,Frozen,Week Name,SP,Subsidiary,Country
0,2024-01-01(W01),202401030846,MNT,OS_AUO_CN_P,EU,EEBN,24GQ50F-B.AEUQ,24GQ50F-BD.CEUVMV,Final26w,N,2024-04-15(W16),572.0,LGEBN,Netherlands
1,2024-01-01(W01),202401030846,MNT,OS_AUO_CN_P,EU,EEBN,27GQ50F-B.AEUQ,27GQ50F-BD.CEUVMV,Final26w,N,2024-02-05(W06),429.0,LGEBN,Netherlands
2,2024-01-01(W01),202401030846,MNT,OS_AUO_CN_P,EU,EEBN,27GQ50F-B.AEUQ,27GQ50F-BD.CEUVMV,Final26w,N,2024-03-18(W12),429.0,LGEBN,Netherlands
3,2024-01-01(W01),202401030846,MNT,OS_AUO_CN_P,EU,EEPL_CZ,24GQ50F-B.AEUQ,24GQ50F-BD.CEUVMV,Final26w,N,2024-03-04(W10),572.0,LGECZ,Czech
4,2024-01-01(W01),202401030846,MNT,OS_AUO_CN_P,EU,EEPL_CZ,27GQ50F-B.AEUQ,27GQ50F-BD.CEUVMV,Final26w,N,2024-02-05(W06),429.0,LGECZ,Czech
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2204,2024-01-01(W01),202401030846,PC,OS_WINGTECH_CN_P,KOREA,EKHQ,10A30Q-L.AQ1UL,10A30Q-L.AQ1UL,Final26w,Y,2024-01-29(W05),1000.0,LGEKR,Korea
2205,2024-01-01(W01),202401030846,PC,OS_WINGTECH_CN_P,KOREA,EKHQ,10A30Q-L.AQ26ML,10A30Q-L.AQ26ML,Final26w,Y,2024-01-29(W05),52.0,LGEKR,Korea
2206,2024-01-01(W01),202401030846,PC,OS_WINGTECH_CN_P,KOREA,EKHQ,10A30Q-L.AQ28K,10A30Q-L.AQ28K,Final26w,Y,2024-01-08(W02),360.0,LGEKR,Korea
2207,2024-01-01(W01),202401030846,PC,OS_WINGTECH_CN_P,KOREA,EKHQ,10A30Q-L.AQ28K,10A30Q-L.AQ28K,Final26w,Y,2024-01-29(W05),228.0,LGEKR,Korea


In [70]:
# sp = sp[sp['From Site'].isin(['OS_TCL_CN_P', 'OS_TPV_CN_P', 'OS_AUO_CN_P'])] # 업체 필터링

# IT ODM 모델명에서 시리즈명 추출
c1 = sp['From Site'].str.contains('AUO|TPV|TCL') # 모니터 ODM 일 경우
c2 = sp['Mapping Model.Suffix'].str.match('\d{2}[T|Z|U|A][G|D|N|W|C]?\d{2}[A-Z]{1,2}') # PC/태블릿 모델일 경우
c3 = sp['Mapping Model.Suffix'].str.contains('CL600') # 페가트론 CL600 모델일 경우

sp.loc[c1, 'Series'] = sp[c1]['Mapping Model.Suffix'].apply(lambda x:x[:2]) # 모니터 모델의 시리즈명(inch) 추출
sp.loc[c2, 'Series'] = sp[c2]['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).str.replace('D|G|N|W|C', '').apply(lambda x:x[:6]) # PC모델의 대표모델명 추출
sp.loc[c3, 'Series'] = 'CL600'

sp[['Year', 'Month']] = sp['Week Name'].apply(lambda x:f'{datetime.date.fromisoformat(x[:10]).isocalendar().year}-{get_month_from_date(datetime.date.fromisoformat(x[:10])):02d}').str.split('-', expand=True)
sp['Quarter'] = sp['Month'].apply(lambda x:pd.to_datetime(x, format='%m')).dt.to_period('Q').astype(str).apply(lambda x:x[-2:])
sp = sp[sp['Week Name'] >= this_week]

C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_1012\1148972203.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  sp.loc[c2, 'Series'] = sp[c2]['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).str.replace('D|G|N|W|C', '').apply(lambda x:x[:6]) # PC모델의 대표모델명 추출


In [91]:
# BR 모델을 MR 모델에 통합
sr.loc[sr['Series'].str.contains('24BR'), 'Series'] = '24MR' 
sr.loc[sr['Series'].str.contains('32BR'), 'Series'] = '32MR'
# 25GR 모델 삭제
sr = sr[~sr['Series'].str.contains('25GR')].reset_index(drop=True)

In [134]:
# Series 모델명 통일화 작업
sp = sp[~sp['Series'].str.contains('25GR')].reset_index(drop=True)
sp.loc[sp['Series'].str.contains('22MS|22BR'), 'Series'] = '22MR'
sp.loc[sp['Series'].str.contains('27MC|27BR'), 'Series'] = '27MR'
sp.loc[sp['Series'].str.contains('32MC|32BR'), 'Series'] = '32MR'
sp.loc[sp['Series'].str.contains('24BR'), 'Series'] = '24MR'

In [71]:
tt = pd.concat([sr, sp], join='inner').reset_index(drop=True)
tt

,Year,Month,Quarter,From Site,Division,Region,Country,Subsidiary,To Site,Series,Mapping Model.Suffix,Week Name,SP
0,2023,08,Q3,OS_TCL_CN_P,MNT,EU,Hungary,LGEMK,EEMK,22,22MR410-B.AEUQ,2023-08-14(W33),840.0
1,2023,08,Q3,OS_TCL_CN_P,MNT,EU,Spain,LGEES,EEES,22,22MR410-B.AEUQ,2023-08-28(W35),838.0
2,2023,08,Q3,OS_TCL_CN_P,MNT,MEA,Turkey,LGETK,EMTK,22,22MR410-B.APDQ,2023-08-28(W35),840.0
3,2023,08,Q3,OS_TCL_CN_P,MNT,SEA,Thailand,LGETH,EATH,22,22MR410-B.ATMQ,2023-08-21(W34),2184.0
4,2023,08,Q3,OS_TCL_CN_P,MNT,SEA,Thailand,LGETH,EATH,22,22MR410-B.ATMQ,2023-08-21(W34),2184.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5726,2024,02,Q1,OS_WINGTECH_CN_P,PC,KOREA,Korea,LGEKR,EKHQ,10A30Q,10A30Q-L.AQ1UL,2024-01-29(W05),1000.0
5727,2024,02,Q1,OS_WINGTECH_CN_P,PC,KOREA,Korea,LGEKR,EKHQ,10A30Q,10A30Q-L.AQ26ML,2024-01-29(W05),52.0
5728,2024,01,Q1,OS_WINGTECH_CN_P,PC,KOREA,Korea,LGEKR,EKHQ,10A30Q,10A30Q-L.AQ28K,2024-01-08(W02),360.0
5729,2024,02,Q1,OS_WINGTECH_CN_P,PC,KOREA,Korea,LGEKR,EKHQ,10A30Q,10A30Q-L.AQ28K,2024-01-29(W05),228.0


In [85]:
tt[tt['Country'] == 'Canada'].pivot_table('SP', index=['From Site', 'Series', 'Mapping Model.Suffix'], columns=['Year', 'Quarter',], aggfunc=sum).loc['OS_TCL_CN_P'].dropna(how='all', axis=1).fillna('-')

Year                           2023    2024       
Quarter                          Q4      Q1     Q2
Series Mapping Model.Suffix                       
22     22MR410-B.AUSQ        1680.0       -  840.0
       22MR41A-B.ACCQ         840.0   840.0      -
       22MR41S-B.ACCQ         840.0  1680.0  840.0
       22MS31W-B.ACCQ             -       -  840.0
27     27MC41D-BS.ACCQ            -  5500.0  750.0
       27MR400-B.AUSQ        2816.0  2304.0  512.0
       27MR41A-B.ACCQ        1280.0  1536.0      -
       27MR41S-B.ACCQ        1280.0  1024.0  512.0

In [29]:
tt['Region'].unique()

array(['EU', 'MEA', 'SEA', 'NA', 'INDIA', 'SCA', 'KOREA'], dtype=object)

In [86]:
region_korean_map = {'NA':'북미', 'EU':'유럽', 'MEA':'중아', 'SEA':'아시아', 'INDIA':'인도', 'SCA':'중남미', 'KOREA':'한국'}

In [87]:
tt['Region'] = tt['Region'].map(region_korean_map)